In [38]:
import torch
import torch.nn as nn
import torch.optim as optim

class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(CBOW, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.fc1 = nn.Linear(embedding_size, vocab_size)

    def forward(self, inputs):
        embeds = self.embedding(inputs)
        sum_embeds = torch.sum(embeds, dim=0)
        out = self.fc1(sum_embeds)
        return out

In [39]:
vocab_size = 130
embedding_size = 20
model = CBOW(vocab_size, embedding_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [58]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)

tensor([73, 73, 63, 63])

In [69]:
def custom_data_generator(data, batch_size):
    """
    A data generator that yields batches of customer data.

    Args:
        data (list): A list of lists of sequential customer purchases.
        batch_size (int): The number of sentences to include in each batch.

    Yields:
        list: A batch of sentences.

    """
    n = len(data)
    while True:
        for i in range(0, n, batch_size):
            yield data[i:i+batch_size][0]

batch_size = 1
generator = custom_data_generator(data, batch_size)


In [70]:
next(generator)

(['Cardigan', 'Cardigan', 'Blouse', 'Blouse'], 'Cardigan')

In [80]:
data = []

for i in range(CONTEXT_SIZE, len(goal) - CONTEXT_SIZE):
    context = (
        [goal[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [goal[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = goal[i]
    data.append((context, target))


In [79]:
# Training loop
for inputs, target in data:
    inputs=make_context_vector(inputs, word_to_ix)
    optimizer.zero_grad()
    outputs = model(inputs)
    target=word_to_ix[target]
    target=torch.tensor(target, dtype=torch.long)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

In [75]:
next(generator)

(['Trousers', 'Trousers', 'Shirt', 'Trousers'], 'Blouse')

In [2]:

enhanced_products_data_path="kaggle_data/initial_data_set.csv"

In [4]:
enhanced_products=pd.read_csv(enhanced_products_data_path)

In [3]:
import pandas as pd

In [42]:
enhanced_products

,1,2,3,4,5,6,7,8,9,10,...,141,142,143,144,145,146,147,148,149,150
0,Cardigan,Cardigan,Cardigan,Blouse,Blouse,Trousers,Trousers,Blouse,Shirt,Trousers,...,Skirt,Trousers,Trousers,Trousers,Shorts,Leggings/Tights,Skirt,Skirt,Blouse,Belt
1,Trousers,Trousers,Trousers,Trousers,Trousers,Trousers,Sweater,T-shirt,Sweater,Sweater,...,Dress,Skirt,Skirt,Sweater,Skirt,Skirt,Skirt,Dress,Swimwear bottom,Swimwear bottom
2,Trousers,Bag,Dress,Bra,Top,Bra,Earring,Trousers,Top,Trousers,...,Jacket,Jacket,Dress,Dress,Top,Skirt,Top,Bra,Top,Top
3,Blouse,Blouse,Blouse,Blouse,Blouse,Blouse,Dress,Top,Blouse,Dress,...,Bag,Bra,Swimwear bottom,Bikini top,Bra,Skirt,Bra,Bikini top,Trousers,Swimwear bottom
4,Trousers,Bag,Bag,Jacket,Dress,Leggings/Tights,Underwear body,Trousers,Shorts,Jumpsuit/Playsuit,...,Top,Skirt,Top,Top,Underwear body,Bra,Bikini top,Dress,Dress,Swimwear bottom
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Jacket,Jacket,Top,Top,Top,Trousers,Trousers,Trousers,Sweater,Top,...,Wedge,Sweater,Leggings/Tights,Sneakers,Trousers,Trousers,Trousers,Dress,Sweater,Sweater
96,Shorts,Shorts,Shorts,Shorts,Shorts,Shorts,Dress,Dress,Blouse,Blouse,...,Jumpsuit/Playsuit,Jumpsuit/Playsuit,Dress,Dress,Bra,Hoodie,Dress,Dress,Night gown,Night gown
97,Ballerinas,Trousers,Blouse,Top,Cardigan,Cardigan,Cardigan,Cardigan,Shirt,Shirt,...,Vest top,Vest top,Blouse,Trousers,Vest top,Trousers,Trousers,Vest top,Shirt,Blouse
98,Dress,Dress,Dress,Leggings/Tights,Leggings/Tights,Leggings/Tights,Sweater,Sweater,Hoodie,Hoodie,...,Dress,Dress,Dress,Dress,Dress,Dress,Dress,Dress,Dress,Dress


In [10]:
list_of_lists = enhanced_products.apply(lambda row: row.tolist(), axis=1).tolist()

In [47]:
import numpy as np

In [51]:
raw_text=np.unique(enhanced_products.values).tolist()

In [29]:
goal=list_of_lists[0]

In [28]:
CONTEXT_SIZE=2

In [52]:
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}

In [56]:
# word_to_ix

[(['Cardigan', 'Cardigan', 'Blouse', 'Blouse'], 'Cardigan'), (['Cardigan', 'Cardigan', 'Blouse', 'Trousers'], 'Blouse'), (['Blouse', 'Cardigan', 'Trousers', 'Trousers'], 'Blouse'), (['Blouse', 'Blouse', 'Trousers', 'Blouse'], 'Trousers'), (['Trousers', 'Blouse', 'Blouse', 'Shirt'], 'Trousers')]


tensor([73, 73, 63, 63])

In [16]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []

for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = (
        [raw_text[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [raw_text[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['are', 'We', 'to', 'study'], 'about'), (['about', 'are', 'study', 'the'], 'to'), (['to', 'about', 'the', 'idea'], 'study'), (['study', 'to', 'idea', 'of'], 'the'), (['the', 'study', 'of', 'a'], 'idea')]


In [61]:
def custom_data_generator(data, batch_size):
    """
    A data generator that yields batches of customer data.

    Args:
        data (list): A list of lists of sequential customer purchases.
        batch_size (int): The number of sentences to include in each batch.

    Yields:
        list: A batch of sentences.

    """
    n = len(data)
    while True:
        for i in range(0, n, batch_size):
            yield data[i:i+batch_size][0]

In [62]:

batch_size = 1
generator = custom_data_generator(list_of_lists, batch_size)

for i in range(2):
    # Do something with the batch of sentences
    print(next(generator))

['Cardigan', 'Cardigan', 'Cardigan', 'Blouse', 'Blouse', 'Trousers', 'Trousers', 'Blouse', 'Shirt', 'Trousers', 'Trousers', 'Shirt', 'Shirt', 'Jacket', 'Jacket', 'Trousers', 'Trousers', 'Cardigan', 'Blouse', 'Shirt', 'Bra', 'Bra', 'Bra', 'Bra', 'Sweater', 'Shorts', 'Shorts', 'T-shirt', 'T-shirt', 'Jumpsuit/Playsuit', 'Skirt', 'Blouse', 'Blouse', 'Blouse', 'Unknown', 'Unknown', 'Sweater', 'Sweater', 'Trousers', 'Leggings/Tights', 'Jacket', 'Jacket', 'Trousers', 'Trousers', 'Dress', 'Jacket', 'Trousers', 'Coat', 'Coat', 'Trousers', 'Trousers', 'Trousers', 'Dress', 'Hair clip', 'Trousers', 'Trousers', 'Vest top', 'Vest top', 'Hat/brim', 'Coat', 'Coat', 'T-shirt', 'T-shirt', 'Trousers', 'Trousers', 'Trousers', 'Trousers', 'Jacket', 'Jacket', 'Jacket', 'Dress', 'Dress', 'Shirt', 'Shirt', 'Jacket', 'Jacket', 'Jacket', 'Trousers', 'Bra', 'Skirt', 'Skirt', 'Jacket', 'Leggings/Tights', 'Trousers', 'Vest top', 'Vest top', 'Vest top', 'Hat/beanie', 'Hat/beanie', 'Jacket', 'Jacket', 'Bra', 'Socks'

In [60]:
next(generator)

[['Trousers',
  'Bag',
  'Dress',
  'Bra',
  'Top',
  'Bra',
  'Earring',
  'Trousers',
  'Top',
  'Trousers',
  'Trousers',
  'Trousers',
  'Trousers',
  'Sneakers',
  'Sneakers',
  'Sneakers',
  'Sneakers',
  'Dress',
  'Skirt',
  'Skirt',
  'Sweater',
  'T-shirt',
  'Top',
  'T-shirt',
  'T-shirt',
  'Top',
  'Top',
  'Dress',
  'Trousers',
  'Cardigan',
  'Leggings/Tights',
  'Trousers',
  'Sweater',
  'Sweater',
  'Top',
  'Dress',
  'Trousers',
  'Trousers',
  'Sweater',
  'Trousers',
  'Cardigan',
  'Trousers',
  'Skirt',
  'Trousers',
  'Sweater',
  'Trousers',
  'Trousers',
  'Dress',
  'Hair/alice band',
  'Top',
  'Gloves',
  'Scarf',
  'Belt',
  'Belt',
  'Underwear Tights',
  'Underwear Tights',
  'Underwear Tights',
  'Dress',
  'Dress',
  'Earring',
  'Dress',
  'Dress',
  'Top',
  'Belt',
  'Belt',
  'Earring',
  'Dress',
  'Jacket',
  'Dress',
  'Bra',
  'Sweater',
  'Pyjama set',
  'Sweater',
  'Trousers',
  'Dress',
  'Dress',
  'Dress',
  'Dress',
  'Trousers',
  'T